### Предобработка
Предобработка в данном решении занимает много процессрного времени на обычном ПК, в репозитории приложен файл concl.csv который содержит конечные результат работы данной предобработки

Структура получившего преобработанного датасета, в качестве колонок содержит список уникальных событий 

События описаны в следующем виде - тип_события+еср_текущей_станции+еср_станции_назначения

В качестве строчек - временные интервалы длиной в 1 час (744 часа месяца)

На пересечении колонки и строки - количество вагонов с которыми произошло данное событие в данный час

In [22]:
#!g1.1
import pandas as pd
import numpy as np
df = pd.read_csv('statistics-07-20.csv', sep=';')

,index_train,length,car_number,destination_esr,adm,danger,gruz,loaded,operation_car,operation_date,...,operation_st_id,operation_train,receiver,rodvag,rod_train,sender,ssp_station_esr,ssp_station_id,tare_weight,weight_brutto
0,NaN,NaN,62827035,984700.0,NaN,NaN,NaN,2.0,81.0,2020-07-16 03:40:00,...,2.000039e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,62827035,862201.0,20.0,NaN,421034.0,NaN,18.0,2020-07-16 14:10:00,...,2.002026e+09,NaN,93149858.0,60.0,NaN,68398528.0,NaN,NaN,249.0,NaN
2,NaN,NaN,62827316,863007.0,NaN,NaN,NaN,2.0,80.0,2020-07-16 14:50:00,...,2.001933e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,62827381,862108.0,NaN,NaN,NaN,2.0,80.0,2020-07-16 14:16:00,...,2.001931e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,62845730,887904.0,20.0,NaN,421034.0,NaN,19.0,2020-07-16 15:15:00,...,2.000039e+09,NaN,81195103.0,60.0,NaN,52682351.0,NaN,NaN,248.0,NaN


Путем ручного анализа событий происходящих с вагоном было установлено, что подмножество некоторых событий имеет цикличность при передвижении вагона в железнодорожной сети. Поэтому выделим эти события для дальнейшего предсказания
Среди всех событий происходящих с вагонами веделим 4
1. DRIVEWAY_EVENT - подача вагона на подъездной путь
2. CHANGE_DESTINATION_EVENT - изменение станции назначения вагона
3. TRAIN_DISPATCH_EVENT - отправление поезда с вагоном
4. ARRIVAL_EVENT - прибытие вагона на станцию

In [31]:
################ Warning! длительная операция! Подумать над оптимизацией
DRIVEWAY_EVENT = 'DRIVEWAY_EVENT'
CHANGE_DESTINATION_EVENT = 'CHANGE_DESTINATION_EVENT'
TRAIN_DISPATCH_EVENT = 'TRAIN_DISPATCH_EVENT'
ARRIVAL_EVENT = 'ARRIVAL_EVENT'

n=1
car_numbers = df['car_number'].unique()

transformed_df = pd.DataFrame(columns=['car_number', 'operation_st_esr',
                                       'destination_esr', 'operation_date', 'event_id'], )
num = 0
for car_number in car_numbers:
    events_with_car = df[df['car_number'] == car_number]
    driveway_events = events_with_car[events_with_car['operation_car'] == 80][['car_number', 'operation_st_esr',
                                       'destination_esr', 'operation_date']]
    event_id = len(driveway_events) * [DRIVEWAY_EVENT]
    driveway_events['event_id'] = event_id
    transformed_df = transformed_df.append(driveway_events)
    # CHANGE_DESTINATION_EVENT
    change_destination_events = events_with_car[events_with_car['destination_esr'].diff().notnull() & events_with_car['destination_esr'].diff() != 0][['car_number', 'operation_st_esr',
                                       'destination_esr', 'operation_date']]
    event_id = len(change_destination_events) * [CHANGE_DESTINATION_EVENT]
    change_destination_events['event_id'] = event_id
    transformed_df = transformed_df.append(change_destination_events)
    # TRAIN_DISPATCH_EVENT
    train_dispatch_events = events_with_car[(events_with_car['operation_train']==2.0)][['car_number', 'operation_st_esr',
                                       'destination_esr', 'operation_date']]
    event_id = len(train_dispatch_events) * [TRAIN_DISPATCH_EVENT]
    train_dispatch_events['event_id'] = event_id
    transformed_df = transformed_df.append(train_dispatch_events)
    #ARRIVAL_EVENT
    arrival_event = events_with_car[events_with_car['operation_st_esr'].diff() != 0][['car_number', 'operation_st_esr',
                                       'destination_esr', 'operation_date']]
    event_id = len(arrival_event) * [ARRIVAL_EVENT]
    arrival_event['event_id'] = event_id
    transformed_df = transformed_df.append(arrival_event)
    num += 1
    if num % 500 == 0:
        print (num/len(car_numbers)*100, '%')
transformed_df.sort_values(by='operation_date')

3.3333333333333335 %
6.666666666666667 %
10.0 %
13.333333333333334 %
16.666666666666664 %
20.0 %
23.333333333333332 %
26.666666666666668 %
30.0 %
33.33333333333333 %
36.666666666666664 %
40.0 %
43.333333333333336 %
46.666666666666664 %
50.0 %
53.333333333333336 %
56.666666666666664 %
60.0 %
63.33333333333333 %
66.66666666666666 %
70.0 %
73.33333333333333 %
76.66666666666667 %
80.0 %
83.33333333333334 %
86.66666666666667 %
90.0 %
93.33333333333333 %
96.66666666666667 %
100.0 %


#### Посчитаем количество вагонов находящихся на конкретной станции 862108 на текущий момент, и используем как feature датасета
TRAIN_DISPATCH_EVENT = (текущее_количество - 1)
ARRIVAL_EVENT = (текущее_количество + 1)

In [ ]:
station = 862108
station_df = transformed_df[transformed_df['operation_st_esr']==station]
count_cars = 0
ts_df = pd.DataFrame(columns=['date', 'count'] )
for row in station_df:
    if row['event_id']== TRAIN_DISPATCH_EVENT:
        count_cars -= 1
        ts_df.append({'date': row['operation_date'], 'count': count_cars})
    if row['event_id']== ARRIVAL_EVENT:
        count_cars += 1
        ts_df.append({'date': row['operation_date'], 'count': count_cars})


Сделаем передескритизацию времени - вместо 1 минуты будет 1 час

In [185]:
count_cars_df = ts_df[['operation_date', 'count']]

In [186]:
def resample(x):
    return x.iloc[len(x)-1]
conversion = {'count': 'last'}

count_cars_df = count_cars_df.set_index("operation_date")
count_cars_df.index = pd.to_datetime(count_cars_df.index, format='%Y-%m-%d %X')
count_cars_df = count_cars_df.resample('H').apply(conversion)
count_cars_df = count_cars_df.reset_index()

In [187]:
count_cars_df.head()

,operation_date,count
0,2020-07-01 00:00:00,4.0
1,2020-07-01 01:00:00,84.0
2,2020-07-01 02:00:00,120.0
3,2020-07-01 03:00:00,152.0
4,2020-07-01 04:00:00,159.0


#### Сформируем полный идентификатор события
Сформируем полный идентификатор события следующим образом -> тип_события+еср_текущей_станции+еср_станции_назначения

In [42]:
#!c1.8
# 
transformed_df['event_full_id'] = transformed_df['event_id'].astype(str) + "_" + transformed_df['operation_st_esr'].astype(str) + "_" + transformed_df['destination_esr'].astype(str)

In [108]:
#!g1.1
## возмем только интересующие колонки тип события и дата
transformed_df = transformed_df[["operation_date", "event_full_id"]]

Сделаем преобразования частоты событий, вместо минуты, возмем промежуток в 1 час и просуммируем количество событий с данным идентификатором за часовой промежуток

In [90]:
def resample(x):
    return x.value_counts().to_json(orient='index')
conversion = {'event_full_id': resample}

transformed_df = transformed_df.set_index("operation_date")
transformed_df.index = pd.to_datetime(transformed_df.index, format='%Y-%m-%d %X')
transformed_df = transformed_df.resample('H').apply(conversion)
transformed_df = transformed_df.reset_index()

In [91]:
transformed_df.columns = transformed_df.columns.get_level_values(0)
transformed_df

,operation_date,event_full_id
0,2020-07-01 00:00:00,"{""TRAIN_DISPATCH_EVENT_889308.0_932809.0"":61,""..."
1,2020-07-01 01:00:00,"{""TRAIN_DISPATCH_EVENT_937906.0_967808.0"":70,""..."
2,2020-07-01 02:00:00,"{""ARRIVAL_EVENT_987708.0_911605.0"":41,""ARRIVAL..."
3,2020-07-01 03:00:00,"{""ARRIVAL_EVENT_920002.0_925701.0"":38,""ARRIVAL..."
4,2020-07-01 04:00:00,"{""ARRIVAL_EVENT_967600.0_937906.0"":74,""TRAIN_D..."
...,...,...
739,2020-07-31 19:00:00,"{""DRIVEWAY_EVENT_831504.0_831504.0"":96,""CHANGE..."
740,2020-07-31 20:00:00,"{""ARRIVAL_EVENT_985906.0_986103.0"":323,""ARRIVA..."
741,2020-07-31 21:00:00,"{""ARRIVAL_EVENT_967600.0_967808.0"":199,""ARRIVA..."
742,2020-07-31 22:00:00,"{""ARRIVAL_EVENT_985906.0_986103.0"":188,""ARRIVA..."


In [166]:
pd.read_json(df['event_full_id'][1], orient='records', typ = 'series')

TRAIN_DISPATCH_EVENT_937906.0_967808.0        70
ARRIVAL_EVENT_937906.0_967808.0               70
TRAIN_DISPATCH_EVENT_911605.0_987801.0        55
ARRIVAL_EVENT_911605.0_987801.0               55
CHANGE_DESTINATION_EVENT_937906.0_967808.0    49
                                              ..
ARRIVAL_EVENT_932809.0_33409.0                 1
ARRIVAL_EVENT_970001.0_865604.0                1
CHANGE_DESTINATION_EVENT_980200.0_980107.0     1
CHANGE_DESTINATION_EVENT_930004.0_982102.0     1
TRAIN_DISPATCH_EVENT_932809.0_33409.0          1
Length: 171, dtype: int64

In [147]:
names_events = set()
for i in range(0, len(df)):    
    names_events = names_events.union(set(pd.read_json(df['event_full_id'][i], orient='records', typ = 'series').index.tolist()))
print(len(names_events))
conc_df = pd.DataFrame(columns=names_events)

60350


Получили 60350 различных уникальных событий

In [ ]:
rows = []
for i in range(0, len(test)):   
    row = pd.read_json(test['event_full_id'][i], orient='records', typ = 'series')
    row = row.reindex(row.index.union(names_events, sort=False), axis=0, fill_value=0,  copy=False)
    rows.append(row)
df_conc = pd.concat(rows, axis=1)

In [162]:
df_conc.head()

,ARRIVAL_EVENT_830003.0_180108.0,ARRIVAL_EVENT_830003.0_233400.0,ARRIVAL_EVENT_830003.0_236305.0,ARRIVAL_EVENT_830003.0_515208.0,ARRIVAL_EVENT_830003.0_525801.0,ARRIVAL_EVENT_830003.0_592204.0,ARRIVAL_EVENT_830003.0_650700.0,ARRIVAL_EVENT_830003.0_655102.0,ARRIVAL_EVENT_830003.0_667805.0,ARRIVAL_EVENT_830003.0_695701.0,...,TRAIN_DISPATCH_EVENT_998100.0_968707.0,TRAIN_DISPATCH_EVENT_998100.0_990005.0,TRAIN_DISPATCH_EVENT_998100.0_990700.0,TRAIN_DISPATCH_EVENT_998100.0_993304.0,TRAIN_DISPATCH_EVENT_998100.0_995102.0,TRAIN_DISPATCH_EVENT_nan_862108.0,TRAIN_DISPATCH_EVENT_nan_864902.0,TRAIN_DISPATCH_EVENT_nan_917207.0,TRAIN_DISPATCH_EVENT_nan_960103.0,TRAIN_DISPATCH_EVENT_nan_971108.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
# Посмотрим одну строчку датасета, выведем фичи, которые не равны
df_conc.iloc[1][df_conc.iloc[1]!=0]

ARRIVAL_EVENT_890004.0_937304.0            1.0
ARRIVAL_EVENT_890108.0_863007.0            3.0
ARRIVAL_EVENT_890108.0_864902.0            1.0
ARRIVAL_EVENT_890108.0_882506.0            1.0
ARRIVAL_EVENT_890108.0_887904.0            3.0
                                          ... 
TRAIN_DISPATCH_EVENT_989309.0_972505.0     1.0
TRAIN_DISPATCH_EVENT_989309.0_974407.0     1.0
TRAIN_DISPATCH_EVENT_989309.0_976600.0     3.0
TRAIN_DISPATCH_EVENT_989309.0_988306.0    13.0
TRAIN_DISPATCH_EVENT_989309.0_989205.0     9.0
Name: 1, Length: 171, dtype: float64

In [189]:
df_conc.join(count_cars_df[['count']]).to_csv('concl.csv', index=False)

In [251]:
## В итоге получаем данный датасет
df = pd.read_csv('concl.csv')


,ARRIVAL_EVENT_830003.0_180108.0,ARRIVAL_EVENT_830003.0_233400.0,ARRIVAL_EVENT_830003.0_236305.0,ARRIVAL_EVENT_830003.0_515208.0,ARRIVAL_EVENT_830003.0_525801.0,ARRIVAL_EVENT_830003.0_592204.0,ARRIVAL_EVENT_830003.0_650700.0,ARRIVAL_EVENT_830003.0_655102.0,ARRIVAL_EVENT_830003.0_667805.0,ARRIVAL_EVENT_830003.0_695701.0,...,TRAIN_DISPATCH_EVENT_998100.0_990005.0,TRAIN_DISPATCH_EVENT_998100.0_990700.0,TRAIN_DISPATCH_EVENT_998100.0_993304.0,TRAIN_DISPATCH_EVENT_998100.0_995102.0,TRAIN_DISPATCH_EVENT_nan_862108.0,TRAIN_DISPATCH_EVENT_nan_864902.0,TRAIN_DISPATCH_EVENT_nan_917207.0,TRAIN_DISPATCH_EVENT_nan_960103.0,TRAIN_DISPATCH_EVENT_nan_971108.0,count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,152.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,159.0
